# Overview

This notebook runs through the week 1 task from the MLX apprenticeship, namely building a language prediction model, and deploying it via fastapi. 

# Imports

In [19]:
import pandas as pd
import torch 

# Solution Structure

- [Proposed dataset](https://huggingface.co/datasets/iix/Parquet_FIles)
- Prep and tokenize dataset for word2vec model
- Create word2vec model
- Train word2vec model to get embedding matrix
- Prep and tokenize dataset for MLP classification model
- Create MLP classification model
- Train MLP classification model
- Write fastapi file, to use pre-trained model, prepping and tokenizing user input
- Launch persistent app via uvicorn/screen

# Prepare Dataset

In [5]:
!wget https://huggingface.co/datasets/iix/Parquet_FIles/resolve/main/Flores7Lang.parquet -O Flores7Lang.parquet

--2023-12-06 09:00:29--  https://huggingface.co/datasets/iix/Parquet_FIles/resolve/main/Flores7Lang.parquet
Resolving huggingface.co (huggingface.co)... 52.84.90.33, 52.84.90.106, 52.84.90.122, ...
Connecting to huggingface.co (huggingface.co)|52.84.90.33|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/fc/9a/fc9a918f6fbc19623c4d012eb54560ce331bef7d7208a8ec162d38c5b4a37971/ef205c73926e8ca0f0cb29276761c4716790d1e15498b0da361063c5348f5e9d?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Flores7Lang.parquet%3B+filename%3D%22Flores7Lang.parquet%22%3B&Expires=1702112429&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMjExMjQyOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9mYy85YS9mYzlhOTE4ZjZmYmMxOTYyM2M0ZDAxMmViNTQ1NjBjZTMzMWJlZjdkNzIwOGE4ZWMxNjJkMzhjNWI0YTM3OTcxL2VmMjA1YzczOTI2ZThjYTBmMGNiMjkyNzY3NjFjNDcxNjc5MGQxZTE1NDk4YjBkYTM2MTA2M2M1MzQ4Z

In [20]:
data = pd.read_parquet("Flores7Lang.parquet")


In [21]:
# Convert dataframe from wide to long format
# Set value_vars to all columns, and no id_vars.
# This creates a new row for every cell in the dataframe, labelled by the column name. 
data_long_format = data.melt(value_vars=data.columns)

In [22]:
# Dump all the translations into one unlabelled list
# The W2V model doesn't need labelled data, that will come in the classification MLP
w2v_corpus = data_long_format['value'].tolist()

In [23]:
# Initialise our W2V torch dataset class, with this corpus
from lang_class_datasets import W2VData
w2v_dataset = W2VData(w2v_corpus)

In [24]:
# Examine some entries, to sanity check
print("word2vec:corpus[0]", w2v_corpus[0])
print("word2vec:ds[0]", w2v_dataset[0])

word2vec:corpus[0] Am Montag haben die Wisenschaftler der Stanford University School of Medicine die Erfindung eines neuen Diagnosetools bekanntgegeben, mit dem Zellen nach ihrem Typ sortiert werden können: ein winziger, ausdruckbarer Chip, der für jeweils etwa einen US-Cent mit Standard-Tintenstrahldruckern hergestellt werden kann.

word2vec:ds[0] (tensor([ 9602, 22262,  1588,  1989]), tensor(12813))


# Store Offline Tokenised Vocab

In [25]:
from tokenizer import Tokenizer
data = pd.read_parquet("./Flores7Lang.parquet")
long_format = data.melt(value_vars=data.columns)
corpus = long_format["value"].tolist()
tknz = Tokenizer(corpus)
tknz.save_vocab("./vocab.txt")
tknz.load_vocab("./vocab.txt")
print(len(tknz.vocab))
print(tknz.vocab[90:100])


26679
['gegenden', 'subir', 'europea', 'lufficio', 'donné', 'joindre', 'telefónico', 'presas', 'libya', 'hotel']


# Create DataLoader, CBOW Model, Loss Function and Optimizer

In [26]:
from model import CBOW
w2v_dataloader = torch.utils.data.DataLoader(w2v_dataset, batch_size=4, shuffle=True)
# CBOW takes (vocab_size, embedding_dim) as inputs
cbow = CBOW(len(w2v_dataset.tokenizer.vocab), 50)
# Use negative log likelihood loss
# In the CBOW task, we're examining the likelihood over the vocab set, in comparison to a ground truth (the missing word)
# This is equivalent to a classification task. Guess the correct class amongst a set of classes 
# Hence, we use negative log likelihood
loss_function = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(cbow.parameters(), lr=0.001)

# Run Training Loop for W2V Model (Logging via Weights and Biases)

In [14]:
import tqdm
n_epochs = 5
for epoch in range(n_epochs):
    total_loss = 0
    for context, target in tqdm.tqdm(w2v_dataloader, desc=f"Epoch {epoch+1}/{n_epochs}", unit="batch"):
        # We don't want to accumulate gradients over batches, so zero them at the start of each batch's training loop
        optimizer.zero_grad()
        # PyTorch calls .forward() automatically, when we pass data to it
        log_probs = cbow(context)
        # Pass ground truth (target) and model outputs (log_probs) to loss function
        loss = loss_function(log_probs, target)
        # Perform backprop step (calc gradients of loss function wrt model params)
        loss.backward()
        # Change the parameter weights, according to selected optimizer
        optimizer.step()
        # Add up loss for printing purposes
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/10, Loss: {total_loss}")
    torch.save(cbow.state_dict(), f"./cbow_epoch_{epoch+1}.pt")

# Prepare Data and DataLoader for Classification MLP Layer

In [37]:
from lang_class_datasets import LangData
# Need data in format (sentence, language_id, language) 
data = pd.read_parquet("./Flores7Lang.parquet")
lang_classification_ds = LangData(data)
lang_classification_dl = torch.utils.data.DataLoader(lang_classification_ds, batch_size=1, shuffle=True)

# Create Classification Model (initialized with W2V weights)

In [38]:
from model import Language
vocab_size = len(lang_classification_ds.tknz.vocab)
state_dict = torch.load(f'cbow_epoch_{n_epochs}.pt')
pre_trained_cbow = CBOW(len(w2v_dataset.tokenizer.vocab), 50)
pre_trained_cbow.load_state_dict(state_dict)
classification_model = Language(pre_trained_cbow.embeddings.weight.data, 7)
# classification_model.load_state_dict(torch.load(f'cbow_epoch_{n_epochs}.pt')))
# torch.save(lang.state_dict(), f"./lang_epoch_0.pt")

# Create Loss Function and Optimizer

In [39]:
# Classification --> use cross entropy
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(classification_model.parameters(), lr=0.001)

# Run Training Loop for Classification Model

In [45]:
n_epochs_classification = 3
for epoch in range(n_epochs_classification):
    for sentence, target, _ in tqdm.tqdm(lang_classification_dl, unit='batch'):
        optimizer.zero_grad()
        log_probs = classification_model(sentence)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
        # print(f"Epoch {epoch+1}/5, Loss: {loss.item()}")
    torch.save(classification_model.state_dict(), f"./classification_model_epoch_{n_epochs_classification+1}.pt")

100%|████████████████████████████████| 14063/14063 [00:11<00:00, 1252.94batch/s]
